In [2]:
!pip install -U -q keras-nlp
!pip install --upgrade keras
!pip install kagglehub

In [3]:
import os
from google.colab import userdata
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [4]:
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [5]:
import os
os.environ["KERAS_BACKEND"] = "jax"

In [6]:
%%capture
import kagglehub

path = kagglehub.model_download("mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf")

In [7]:
print("Path to model files:", path)

Path to model files: /root/.cache/kagglehub/models/mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf/1


In [8]:
!pip install bitsandbytes peft trl --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00


In [9]:
from datasets import load_dataset

In [10]:
data = load_dataset("valerielucro/gsm8k_preference_dataset",split="train")

README.md:   0%|          | 0.00/449 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/721k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/188k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/716 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/179 [00:00<?, ? examples/s]

In [11]:
def preprocess(data):
    data['prompt'] = '<s>[INST]' + data['prompt'] + '\n do it step by step [/INST]'
    data['chosen'] += '</s>'
    data['rejected'] += '</s>'
    return data

In [12]:
train_dataset = data.map(preprocess)

Map:   0%|          | 0/716 [00:00<?, ? examples/s]

In [14]:
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.8/313.8 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00


In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model,PeftModel
from trl import DPOTrainer,DPOConfig
import torch
import wandb

In [17]:
from google.colab import userdata

wandb_token = userdata.get('wandb')
HF_token = userdata.get('HF_TOKEN')

In [18]:
from huggingface_hub import login
login(HF_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [19]:
dataset_size = len(train_dataset)
num_of_epochs = 1

In [20]:
notes = f"""
initial DPO test run on sample gsm8k preference dataset of {dataset_size} and {num_of_epochs} epochs"""

In [21]:
wandb.login(key = wandb_token)
run = wandb.init(
    project='gsm8k',
    job_type="training",
    name="test run with DPO"
    ,notes = notes
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sosamaali (midconstruct). Use `wandb login --relogin` to force relogin


In [22]:
base_model = "/root/.cache/kagglehub/models/mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf/1"

In [23]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [24]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

In [25]:
peft_path = "valerielucro/mistral_gsm8k_sft_cot"

In [26]:
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto"
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
model = PeftModel.from_pretrained(model,peft_path)
model = prepare_model_for_kbit_training(model)

adapter_config.json:   0%|          | 0.00/753 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

In [28]:
ref_model = PeftModel.from_pretrained(model,peft_path)
ref_model = prepare_model_for_kbit_training(ref_model)

In [29]:
training_args = DPOConfig(
    output_dir="/content/working/checkpoints",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    beta=0.2
)

In [30]:
trainer = DPOTrainer(
    model,
    ref_model,
    train_dataset=train_dataset,
    args=training_args,
    tokenizer=tokenizer
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:680: UserWarning: `max_length` is not set in the DPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:693: UserWarning: `max_prompt_length` is not set in the DPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:728: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/716 [00:00<?, ? examples/s]

In [31]:
trainer.train()
wandb.finish()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please u

Step,Training Loss


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:243: UserWarning: Could not find a config file in /kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(


train/epoch,▁
train/global_step,▁
total_flos,0
train/epoch,1
train/global_step,179
train_loss,0.69315
train_runtime,4812.7149
train_samples_per_second,0.149
train_steps_per_second,0.037


In [32]:
fine_tuned_model_name = "mistral_7B_sft_and_dpo"

In [33]:
trainer.model.save_pretrained(fine_tuned_model_name)

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:243: UserWarning: Could not find a config file in /kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(


In [34]:
commit_message = "initial adapter with SFT+DPO on gsm8k preference dataset and 1 epoch"

In [35]:
trainer.model.push_to_hub(fine_tuned_model_name,commit_message=commit_message)
tokenizer.push_to_hub(fine_tuned_model_name, commit_message=commit_message)

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/OsamaAliMid/mistral_7B_sft_and_dpo/commit/4f8e1f7f20f4c3773b06e938ab020b0bf19c9e46', commit_message='initial adapter with SFT+DPO on gsm8k preference dataset and 1 epoch', commit_description='', oid='4f8e1f7f20f4c3773b06e938ab020b0bf19c9e46', pr_url=None, pr_revision=None, pr_num=None)